# Linear Algebra Algorithms

## Implementation

In [1]:
import numpy as np
import math

### Property checks

#### Simetric check

In [2]:
def is_simetric(matrix):
    order = matrix.shape[0]
    
    for i in range(order):
        for j in range(order):
            if matrix[i,j] != matrix[j,i]:
                return False
 
    return True

#### Simetric and positive defined check

In [3]:
def is_simetric_positive_defined(matrix):
    order = matrix.shape[0]
    
    for i in range(order):
        if matrix[i,i] < 0:
            return False
        
        for j in range(order):
            if matrix[i,j] != matrix[j,i]:
                return False
    
    return True

#### Diagonal dominance check

In [1]:
def is_diagonal_dominant(A):
    order = A.shape[0]
    
    for i in range(order):
        row_sum = 0
        column_sum = 0
        
        for j in range(order):
            if (i == j):
                continue
            row_sum += abs(A[i, j])
            column_sum += abs(A[j, i])
                    
        if (abs(A[i, i]) < row_sum or abs(A[i, i]) < column_sum):
            return False
        
    return True

### Decomposition

#### LU Decomposition

In [5]:
def pivotting_matrix_step(A, k):
    order = A.shape[0]
    P = np.identity(order)
    
    # No pivotting needed
    if (A[k,k] != 0):
        return P
        
    # Find useful row and swap
    for i in range(k + 1, order - 1):
        if A[i,k] != 0:
            temp = np.array(P[k,:])
            P[k,:] = P[i,:]
            P[i,:] = temp
            return P
    
    return None

def lu_decomposition_step(A, k):        
        order = A.shape[0]
                
        L = np.identity(order)
        U = np.identity(order)
        
        for i in range (k + 1, order):
            element = A[i,k] / A[k,k]
            L[i, k] = element
            U[i, k] = - element
            
        return L, U
    
def lu_decompose(A):
    order = A.shape[0]
    
    U = A
    Mu_combined = np.identity(order)
    L = np.identity(order)
    P_combined = np.identity(order)
    
    for k in range (order - 1):
        P = pivotting_matrix_step(U, k)
        
        if (P is None):
            return -1
        
        P_combined = P @ P_combined
        U = P @ U
        
        Ml, Mu = lu_decomposition_step(U, k)
        Mu_combined = Mu @ Mu_combined
        
        L = Ml @ L
        U = Mu @ U

    return L, U, Mu_combined, P_combined

#### Cholesky decomposition

In [6]:
def cholesky_decompose(A):
    order = A.shape[0]

    L = np.zeros((order, order))

    for i in range(order):
        for k in range(i+1):
            subtrahend = sum(L[i,j] * L[k,j] for j in range(k))
            
            if (i == k):
                L[i,k] = math.sqrt(A[i,i] - subtrahend)
            else:
                L[i,k] = (1.0 / L[k,k] * (A[i,k] - subtrahend))
    return L

#### Gauss elimination

In [7]:
def gauss_elimination(A):
    _, U, Mu_combined, P = lu_decompose(A)
    return U, Mu_combined, P

#### Determinant

In [8]:
def determinant(A):
    order = A.shape[0]
    L, U, _, _ = lu_decompose(A)
    
    l_det = 1
    u_det = 1
    
    for index in range(order):
        l_det = L[index, index] * l_det
        u_det = U[index, index] * u_det
        
    return l_det * u_det

### AX = B solving

#### By gauss elimination

In [9]:
def solve_gauss_elimination(A, B):
    order = A.shape[0]

    A, M, P = gauss_elimination(A)
    B = M @ P @ B
    X = np.zeros(order)
         
    for i in range(order - 1, -1, -1): # From n to 0 
        subtrahend = sum(A[i,j] * X[j] for j in range(i + 1, order)) 
        X[i] = (B[i] - subtrahend) / A[i, i]
        
    return X

#### By Jacobi method

In [10]:
def solve_jacobi(A, B, rtol=0.000001):
    order = A.shape[0]
    residue = rtol + 1
    X = np.ones(order)
    
    if (not is_diagonal_dominant(A)):
        return -1
        
    while(residue > rtol):
        X_current = np.ones(order)
        
        for i in range (order):
            subtrahend = 0
            
            for j in range (order):
                if (j == i):
                    continue
                subtrahend += A[i, j] * X[j]
            
            X_current[i] = (B[i] - subtrahend) / A[i, i]
        
        residue = np.linalg.norm(X_current - X, ord=2) / np.linalg.norm(X_current, ord=2)
        X = X_current
    
    return X

#### By Gauss-seidel method

In [11]:
def solve_gauss_seidel(A, B, rtol=0.000001):
    order = A.shape[0]
    residue = rtol + 1
    X = np.ones(order)
    
    if (not is_simetric_positive_defined(A) and not is_diagonal_dominant(A)):
        return -1
    
    while(residue > rtol):
        X_current = np.ones(order)
        
        for i in range (order):
            subtrahend = 0
            
            for j in range (i):
                subtrahend += A[i, j] * X_current[j]
    
            for j in range (i + 1, order):
                subtrahend += A[i, j] * X[j]

            X_current[i] = (B[i] - subtrahend) / A[i, i]
        
        residue = np.linalg.norm(X_current - X, ord=2) / np.linalg.norm(X_current, ord=2)
        X = X_current
    
    return X

### Eigenvalues and eigenvectors

#### Power Method

Returns the greatest eigenvector and its corresponding eigenvalue

In [12]:
def eigen_power_method(A, rtol=0.000001):
    order = A.shape[0]
    
    eigenvector = np.ones(order)
    residue = rtol + 1
    prev_eigenvalue = 1
    
    while(rtol < residue):   
        Y = A @ eigenvector
        
        eigenvalue = Y[0]
        eigenvector = Y / eigenvalue
        
        residue = (eigenvalue - prev_eigenvalue) / eigenvalue
        prev_eigenvalue = eigenvalue

    return eigenvector, eigenvalue 

#### Jacobi method

Returns aproximated eigenvalues and eigenvectors. Matrix Mu_combinedst be simetric.

In [13]:
def eigen_jacobi(A, rtol=0.00000001):
    order = A.shape[0]
    
    if not is_simetric(A):
        return -1
        
    X = np.identity(order)
    residue = rtol + 1
    
    while(residue > rtol):
        # Finds greater absolute value position outside diagonal
        greatest_el = 0
        for i in range(order):
            for j in range(order):
                if i != j and abs(A[i,j]) > abs(greatest_el):
                    greatest_el_pos = (i, j)
                    greatest_el = A[i,j]
        
        # Compute phi for P matrix
        i, j = greatest_el_pos
        if (A[i,i] != A[j,j]):
            phi = 1/2 * np.arctan(2 * A[i,j] / (A[i,i] - A[j,j]))
        else:
            phi = np.pi / 4
        
        # Compute P matrix
        P = np.identity(order)
        P[i,i] = np.cos(phi)
        P[j,j] = np.cos(phi)
        P[j,i] = np.sin(phi)
        P[i,j] = -np.sin(phi)
        
        # Next iteration A matrix
        residue = abs(greatest_el)
        A = np.transpose(P) @ A @ P
        X = X @ P
        
    eigenvalues = np.zeros(order)
    
    # Eigenvalues are the values in the main diagonal
    for i in range(order):
        for j in range(order):
            if i == j:
                eigenvalues[i] = A[i,j]

    eigenvectors = X 
    
    return eigenvectors, eigenvalues

### Linear regression

#### Least squares

In [14]:
def least_squares(points):
    n_points = points.shape[0]
    
    A = np.identity(2)
    A[0,0] = n_points
    A[0,1] = sum(points[i,0] for i in range (n_points))
    A[1,0] = A[0,1]
    A[1,1] = sum((points[i,0] ** 2) for i in range (n_points))
        
    C = np.array([0.,0.])
    C[0] = sum(points[i,1] for i in range (n_points))
    C[1] = sum(points[i,0] * points[i,1] for i in range (n_points))
            
    B = solve_gauss_seidel(A, C)
    
    return B